In [3]:
import os, base64

directory="images"
index="000000"

image_data = []
for folder in os.listdir(directory):
    if not folder.startswith("cam"): continue
    for filename in os.listdir(f"{directory}/{folder}"):
        if filename.startswith(index) and "Distance" not in filename:
            with open(f"{directory}/{folder}/{filename}", 'rb') as f:
                image_data.append(base64.b64encode(f.read()).decode('utf-8'))

In [8]:
for folder in os.listdir(os.getcwd()):
    if os.path.isdir(folder) and folder != '.git': print(folder)

images
